## Residual CNN from https://arxiv.org/abs/1805.00794
This model is an implementation of the one presented in the paper.

The model relies on repeated 1D convolutions to build lower dimensional representations of the output. 
The structure's main feature is the repetition of residual blocks. These are made up of two Convolutional layers, a residual connection from the input to the block, ReLU and MaxPooling.

The dimensionality reduction is carried out using MaxPool layers, and each Conv layer maintains the same number of filters (32), which is somewhat unusual.

The residual networks aid in backpropagation of the gradient, however, this model is perhaps not quite deep enough to warant these, as shown by the performance of the [Baseline CNN](../baseline/baseline_cnn_mitbih.ipynb), which is essentially this model without the residual connections.

From the performance of the [Deep Residual CNN](../deep_res/cnn_res_deep_mitbih.ipynb), we also see that we can better exploit these residual connections by making the model much deeper.

![Network](cnn_paper.png)

In [1]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, Add, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

### MITBIH baseline
#### Code mostly from https://github.com/CVxTz/ECG_Heartbeat_Classification/blob/master/code/baseline_mitbih.py

In [2]:
df_train = pd.read_csv("../data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("../data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

### Load model and skip to [evaluation](#evaluate)

In [3]:
model = load_model("cnn_paper_mitbih.h5")

In [9]:
img = plot_model(model, to_file="cnn_paper.png", show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


### Or ALTERNATIVELY train it

In [ ]:
def maxpool_block(X, filters, kernel_size=5, dropout=0.1, pool_size=2):
    img_1 = Convolution1D(filters, kernel_size=kernel_size, activation='relu', padding='same')(X)
    img_1 = Convolution1D(filters, kernel_size=kernel_size, activation='relu', padding='same')(img_1)
    img_1 = Add()([X, img_1])
    img_1 = Activation('relu')(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    return img_1

In [ ]:
def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(32, kernel_size=5, activation=activations.relu, padding="valid")(inp)     
    for i in range(5):
        img_1 = maxpool_block(img_1, 32, dropout=0.2) 
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="same")(img_1)  
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="same", name="final_conv")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(32, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(32, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])      
    model.summary()
    return model

In [ ]:
model = get_model()
file_path = "cnn_paper_mitbih_retrain.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

### Evaluation
<a id='evaluate'></a>

In [4]:
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

print(confusion_matrix(
    Y_test, pred_test))

Test f1 score : 0.9270679431082358 
Test accuracy score : 0.9874383336378586 
[[18043    40    21     4    10]
 [   88   460     7     0     1]
 [   27     5  1395    17     4]
 [   19     1    14   127     1]
 [   12     0     4     0  1592]]
